<a href="https://colab.research.google.com/github/aj2622/reinforcement-learning/blob/master/KhunPoker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import random 

< Khun Poker Definintions >

In [0]:
PASS = 0
BET = 1
NUM_ACTIONS = 2

< Information set node class definition >

In [0]:
class Node:
  def __init__(self,infoSet):
    self.num_actions = 2
    self.strategy = np.zeros(self.num_actions)
    self.regretSum = np.zeros(self.num_actions)
    self.strategySum = np.zeros(self.num_actions)
    self.infoSet = infoSet
    
  def getStrategy(self,realizationWeight):
    normalizingSum = 0
    self.strategy = np.maximum(self.regretSum,0)
    normalizingSum = np.sum(self.strategy)
    if normalizingSum > 0:
      self.strategy /= normalizingSum
    else:
      self.strategy = np.array([1/self.num_actions for i in range(self.num_actions)])
    self.strategySum += self.strategy * realizationWeight
    return self.strategy
  
  def getAverageStrategy(self):
    avgStrategy = np.zeros(self.num_actions)
    normalizingSum = 0
    normalizingSum = np.sum(self.strategySum)
    if normalizingSum > 0:
      avgStrategy = self.strategySum / normalizingSum
    else:
      avgStrategy = [1/self.num_actions for i in range(self.num_actions)]
    return avgStrategy

< Counterfactual Regret Minimization Iteration >

In [0]:
def cfr(cards, history, p0, p1):
  global infoSetDic, NUM_ACTIONS
  plays = len(history)
  player = plays % 2
  opponent = 1 - player
  
  #  <return payoff for terminal satate>
  if plays > 1:
    terminalPass = history[plays - 1] == 'p'
    doubleBet = history[plays-2:plays] == 'bb' 
    isPlayerHigherCard = cards[player] > cards[opponent]
    if terminalPass:
      if history == 'pp':
        if isPlayerHigherCard:
          return 1
        else:
          return -1
      else:
        return 1
    elif doubleBet:
      if isPlayerHigherCard:
        return 2
      else:
        return -2
      
  infoSet = str(cards[player]) + history
  if infoSet in infoSetDic:
    node = infoSetDic[infoSet]
  else:
    infoSetDic[infoSet] = Node(infoSet)
    node = infoSetDic[infoSet]
  
  if player == 0:
    realizationWeight = p0
  else:
    realizationWeight = p1
  
  strategy = node.getStrategy(realizationWeight)
  util = np.zeros(NUM_ACTIONS)
  nodeUtil = 0
  for a in range(NUM_ACTIONS):
    if a == 0:
      nextHistory = history + 'p'
    else:
      nextHistory = history + 'b'
    if player == 0:
      util[a] = -cfr(cards, nextHistory, p0*strategy[a], p1)
    else:
      util[a] = -cfr(cards, nextHistory, p0, p1*strategy[a])
    nodeUtil += strategy[a] * util[a]
    
  for a in range(NUM_ACTIONS):
    regret = util[a] - nodeUtil
    if player == 0:  
      node.regretSum[a] += p1 * regret
    else:
      node.regretSum[a] += p0 * regret
      
  return nodeUtil

< Train Khun Poker >

In [0]:
def train(iterations):
  cards = [1,2,3]
  util = 0
  for i in range(iterations):
    random.shuffle(cards)
    util += cfr(cards,'',1,1)
  print("Avrage game value is {}".format(str(util/iterations)))
  for i in infoSetDic:  
    print('{}: {}'.format(infoSetDic[i].infoSet,infoSetDic[i].getAverageStrategy()))

In [0]:
infoSetDic = {}
train(1000000)

Avrage game value is -0.05527093672666206
2: [9.99994384e-01 5.61614334e-06]
3p: [1.50191193e-06 9.99998498e-01]
2pb: [0.42329679 0.57670321]
3b: [1.50191193e-06 9.99998498e-01]
3: [0.27676106 0.72323894]
1p: [0.67028535 0.32971465]
3pb: [2.71534677e-06 9.99997285e-01]
1b: [9.99998503e-01 1.49656688e-06]
1: [0.75918339 0.24081661]
1pb: [9.99999013e-01 9.87486876e-07]
2p: [9.99998498e-01 1.50153307e-06]
2b: [0.66566226 0.33433774]
